<a href="https://colab.research.google.com/github/Sulaiman29/BDA_HW5/blob/main/Copy_of_BDA2025Spring_HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div class="alert alert-block alert-info"><b></b>
<h1><center> <font color='black'> Homework 05</font></center></h1>
<h2><center> <font color='black'> A/B Testing & Uplift modelling</font></center></h2>    
<h2><center> <font color='black'> MTAT.03.319 - Business Data Analytics</font></center></h2>
<h2><center> <font color='black'> University of Tartu</font></center></h2>
</div>

# Homework instructions

- Please provide the names and student IDs of the team-members (Maximum 2 person) in the field "Team mates" below. If you are not working in a team please insert only your name and student ID.

- Please provide code where ever applicable.

- The accepted submission format is .ipynb file. If you are sharing Colab link, make sure that the privacy settings for the file is public so we can access your code.

- The submission will automatically close on <font color='red'>**18 May at 23:59**</font>, so please make sure to submit before the deadline.

- ONLY one of the teammates should submit the homework and in the submission description the other person's Name and Student ID must be entered. We will grade the homework and the marks and feedback is applied for both the team members. So please communicate with your team member about marks and feedback if you are submit the homework.

- If a question is not clear, please ask us in Moodle ONLY.

- After you have finished solving the Homework, please restart the Kernel and run all the cells to check if there is any persisting issues.

- Plagiarism is <font color='red'>**PROHIBITED**</font>. Any form of plagiarism will be dealt according to the university policy (https://ut.ee/en/content/academic-fraud).

- <font color='red'>**DO NOT CHANGE THE TEMPLATE**</font>

- <font color='red'>**Restart the Kernel and Run all the cells once again after you are done.**</font>
This will ensure that all the cells run without error. You will find an option in the top menu bar under Kernel tab.

**<h2><font color='red'>Team mates:</font></h2>**


<font color='red'>Name: </font>&emsp;   <font color='red'>Student ID: </font>


<font color='red'>Name: </font>&emsp;   <font color='red'>Student ID: </font>



### The homework is divided into two sections and the points are distributed as below:
<pre>
- A/B Testing               -> 5.5 points
- Uplift modeling           -> 5.5 points
________________________________________________
Total                       -> 11.0 points
</pre>

# 1. A/B Testing (5.5 points)


**1.1 Use `AB_clicks.csv` data and find whether Learn, Help, and Services versions of the page compared to the Interact have significantly more (or less) clicks (compare distributions between versions). Justify the choice of the performed tests and interpret the result of the test. (1.5 points)**


**1.2 Use two different methods to test the normality of two different versions of your choice (Connect, Help, Services,..) (2.0 points)**





**1.3 Read about Multiple comparison problem read ([wikipedia](https://en.wikipedia.org/wiki/Multiple_comparisons_problem), [An Overview of Methods to Address the Multiple Comparison Problem](https://towardsdatascience.com/an-overview-of-methods-to-address-the-multiple-comparison-problem-310427b3ba92) or other resources). What is the problem and when we need to do something about it (describe briefly)? (1 points)**

<font color='red'> **Answer:**

**1.4 Load the dataset classifier_results.csv. The dataset contains the following fields:**

**Dataset** - 200 datasets retreived from [OpenML](https://www.openml.org/search?type=data) dataset directory.

**Classifier** - 9 classifiers from scikit-learn

**Accuracy** - The accuracy achieved after applying the classifier to the dataset


**Is there a significant statistical difference between these classifiers ? If you were asked to continue your analysis with only a limited number of classifiers how many would you pick and why ? (1.0 points)**

<font color='red'> **Answer:**

# 2. Uplift modelling (5.5 points)
For this task we are going to use email_marketing.csv from [MineThatData](https://blog.minethatdata.com/2008/03/minethatdata-e-mail-analytics-and-data.html). The data was recorded from a two-weeks email campaign that included 64000 customers. Out of these customers:
1. 1/3 were randomly chosen to receive an e-mail campaign featuring Mens merchandise.
2. 1/3 were randomly chosen to receive an e-mail campaign featuring Womens merchandise.
3. 1/3 were randomly chosen to not receive an e-mail campaign.

The features in this dataset include:

- Recency: Months since last purchase.
- History: Actual dollar value spent in the past year.
- Mens: 1/0 indicator, 1 = customer purchased Mens merchandise in the past year.
- Womens: 1/0 indicator, 1 = customer purchased Womens merchandise in the past year.
- Zip_Code: Classifies zip code as Urban, Suburban, or Rural.
- Newbie: 1/0 indicator, 1 = New customer in the past twelve months.
- Channel: Describes the channels the customer purchased from in the past year.
- Segment: e-mail campaign  type that the customer received
- Visit: 1/0 indicator, 1 = Customer visited website in the following two weeks.
- Spend: Actual dollars spent in the following two weeks.

The Conversion variable is realted to the evaluation if the campaign was effective or not.

- Conversion: 1/0 indicator, 1 = Customer purchased merchandise in the following two weeks.


**2.0 Load the dataset and provide description. (0.25 points)**

**2.1 Use the function calc_uplift() from the practice session to calculate the current uplift of the campaign. Remeber to modify the function according to the need. (0.5 points)**

**2.2 Create a new column named target_class. Split the customers in 4 categories:**

1- Treatment Responders

2- Treatment Non-Responders (sleeping dogs)

3- Control Responders (sure things)

4-Control Non-Responders(lost causes).

**Label encode these categories and store the result for each customer in the column target_class (0.5 points)**

**2.3 Apply the necessary preprocessing to the data and split it into train and test set, using 80/20 ratio. Remember that you have to drop the columns that define the label and perform preprocessing. Build a model with your classifier of choice that will predict probabilities for the categories mentioned in 2.2. (1.5 points)**

**2.4 Calculate the uplift score using probabilities for all customers and store the results to a new column in the dataframe (0.5 points)**

**2.5  Select the customers with High Uplift Score as below:**
    
**High Uplift Score:** Customers have uplift score > 3rd quantile

**a. Calculate the ratio of the customers targeted with Womens e-mail that have high uplift score over the total customers who received emails featuring Womens merchandise. (0.5 points)**

**b. Calculate the ratio of the customers targeted with Mens e-mail that have high uplift score over the total customers who received emails featuring Mens merchandise. (0.5 points)**


**c.Which one of the campaigns would you say is more successful ? (0.5 points)**


<font color='red'> **Answer C:**

**2.6 For customers in the segment named "Womens e-mail" with high uplift score calculate conversion uplift and revenue uplift. Compare it with the benchmark from excercise 2.1 and draw conclusions. (0.75 points)**

<font color='red'> **Answer:**

## How long did it take you to solve the homework?

* Please answer as precisely as you can. It does not affect your points or grade in any way. It is okay, if it took 0.5 hours or 24 hours. The collected information will be used to improve future homeworks.

<font color='red'> **Answer: X hours**</font>


## What is the level of difficulty for this homework?
you can put only number between $0:10$ ($0:$ easy, $10:$ difficult)

<font color='red'> **Answer:**</font>

## Any other comments regarding the homework?

<font color='red'> **Answer:**</font>